<a href="https://colab.research.google.com/github/tngus4334/LG-Aimers/blob/main/Y_06_Pycaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --pre pycaret
!pip install xgboost==1.1.0
!pip install catboost

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 7.4 MB/s 
     |████████████████████████████████| 81 kB 5.4 MB/s 
     |████████████████████████████████| 209 kB 31.8 MB/s 
     |████████████████████████████████| 78 kB 5.9 MB/s 
     |████████████████████████████████| 147 kB 7.1 MB/s 
     |████████████████████████████████| 49 kB 4.5 MB/s 
     |████████████████████████████████| 112 kB 62.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=38641e641b934e570c91c2bd8cc8cd99a4c447d308bf95cd90a7804645cd6040
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline


import time
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

from sklearn.preprocessing import Normalizer
from sklearn.multioutput import MultiOutputRegressor

from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from sklearn.metrics import mean_absolute_error
import random
#import shap
#import optuna

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import ExtraTreeRegressor

import random
import os

import xgboost as xgb

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/데이콘/자율주행센서/train.csv')
test = pd.read_csv('/content/drive/MyDrive/데이콘/자율주행센서/test.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/데이콘/자율주행센서/sample_submission.csv')

In [ ]:
train.drop(['X_02','X_04','X_10','X_11','X_23','X_47','X_48'],axis=1,inplace=True)
test.drop(['X_02','X_04','X_10','X_11','X_23','X_47','X_48'],axis=1,inplace=True)

### train 전처리

In [ ]:
# 스크류 삽입깊이 단차
train['max-min_X'] = train[['X_19','X_20','X_21','X_22']].max(axis=1) - train[['X_19','X_20','X_21','X_22']].min(axis=1)

In [ ]:
# 하우징 기울기
train['slope_X'] = train[['X_38','X_39','X_40']].max(axis = 1) - train[['X_38','X_39','X_40']].min(axis = 1)

In [ ]:
# 납량차이
train['SMT_X'] = train[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].max(axis = 1) - train[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].min(axis = 1)

In [ ]:
# 비슷한 종류들끼리 평균
train['X1~X6'] =  train[['X_01','X_06']].mean(axis=1)
train['X14~X18'] = train[['X_14','X_15','X_16','X_17','X_18']].mean(axis=1)
train['X19~X22'] = train[['X_19','X_20','X_21','X_22']].mean(axis=1)
train['X24~X29'] = train[['X_24','X_25','X_26','X_27','X_28','X_29']].mean(axis=1)
train['X30~X32'] = train[['X_30','X_32']].mean(axis=1)
train['X31~X33'] = train[['X_31','X_33']].mean(axis=1)
train['X38~X40'] = train[['X_38','X_39','X_40']].mean(axis=1)
train['X41~X45'] = train[['X_41','X_42','X_43','X_44','X_45']].mean(axis=1)
train['X50~X56'] = train[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].mean(axis=1)

In [ ]:
# 단차
train['con1_X'] = train['X_12'] - train['X_24']
train['con2_X'] = train['X_12'] - train['X_25']
train['con3_X'] = train['X_12'] - train['X_26']
train['con4_X'] = train['X_12'] - train['X_27']
train['con5_X'] = train['X_12'] - train['X_28']
train['con6_X'] = train['X_12'] - train['X_29']

In [ ]:
# 중복으로 넣기
train['nd_area_X1'] = train['X_08']*train['X_07']/train['X_03']
train['th_area_X1'] = train['X_09']*train['X_07']/train['X_03']

train['nd_area_X2'] = train['X_08']*train['X_07']/train['X_03']
train['th_area_X2'] = train['X_09']*train['X_07']/train['X_03']

In [ ]:
train = train.replace(train['X_24'].min(), train['X_24'].mean())

In [ ]:
# 무게 / 면적
train['X3/X7'] = train['X_03']/ train['X_07']

# 스크류 깊이 평균
train['X19X30'] = train[['X_19','X_30']].mean(axis=1)
train['X20X31'] = train[['X_20','X_31']].mean(axis=1)
train['X21X32'] = train[['X_21','X_32']].mean(axis=1)
train['X22X33'] = train[['X_22','X_33']].mean(axis=1)

In [ ]:
# 표준편차
train['X14~X18_std'] = train[['X_14', 'X_15', 'X_16', 'X_17', 'X_18']].std(axis=1)
train['X19~X22_std'] = train[['X_19','X_20','X_21','X_22']].std(axis=1)
train['X24~X29_std'] =  train[['X_24', 'X_25', 'X_26','X_27','X_28','X_29']].std(axis=1)

train['X38~X40_std'] = train[['X_38','X_39','X_40']].std(axis=1)

train['X41~X45_std'] = train[['X_41','X_42', 'X_43', 'X_44', 'X_45']].std(axis=1)
train['X50~X56_std'] = train[['X_50','X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56']].std(axis=1)

In [ ]:
# 중앙값
train['X14~X18_mid'] = train[['X_14', 'X_15', 'X_16', 'X_17', 'X_18']].median(axis=1)
train['X19~X22_mid'] = train[['X_19','X_20','X_21','X_22']].median(axis=1)
train['X24~X29_mid'] =  train[['X_24', 'X_25', 'X_26','X_27','X_28','X_29']].median(axis=1)

train['X38~X40_mid'] = train[['X_38','X_39','X_40']].median(axis=1)

train['X41~X45_mid'] = train[['X_41','X_42', 'X_43', 'X_44', 'X_45']].median(axis=1)
train['X50~X56_mid'] = train[['X_50','X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56']].median(axis=1)

In [ ]:
# 튀는값 평균으로 대치
mean_X_30 = train[(train['X_30']<1.7) & (train['X_30']>1)]['X_30'].mean()
mean_X_31 = train[(train['X_31']<5)]['X_31'].mean()
mean_X_32 = train[(train['X_32']<1.7) & (train['X_32']>1)]['X_32'].mean()
mean_X_33 = train[(train['X_31']<5)]['X_31'].mean()
mean_X_38 = train[(train['X_33']<0.7)]['X_33'].mean()

In [ ]:
train['X_30'] = train['X_30'].apply(lambda x : mean_X_30 if x >1.7 or x < 1 else x)
train['X_31'] = train['X_31'].apply(lambda x : mean_X_31 if x > 5 else x)
train['X_32'] = train['X_32'].apply(lambda x : mean_X_32 if x >1.7 or x < 1 else x)
train['X_33'] = train['X_33'].apply(lambda x : mean_X_33 if x > 5 else x)
train['X_38'] = train['X_38'].apply(lambda x : mean_X_38 if x > 0.7 else x)

In [ ]:
# 공차
train['gongcha_1_X'] = train[['X_38','X_39','X_40']].sum(axis = 1)
train['gongcha_2_X'] = train[['X_24','X_25','X_26','X_27','X_28','X_29']].sum(axis = 1)

In [ ]:
train['all_sum_X'] = train.sum(axis=1)
train['all_mean_X'] = train.mean(axis=1)

### test 전처리

In [ ]:
test['max-min_X'] = test[['X_19','X_20','X_21','X_22']].max(axis=1) - test[['X_19','X_20','X_21','X_22']].min(axis=1)

In [ ]:
# 하우징 기울기
test['slope_X'] = test[['X_38','X_39','X_40']].max(axis = 1) - test[['X_38','X_39','X_40']].min(axis = 1)

In [ ]:
# 납량차이
test['SMT_X'] = test[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].max(axis = 1) - test[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].min(axis = 1)

In [ ]:
test['X1~X6'] =  test[['X_01','X_06']].mean(axis=1)
test['X14~X18'] = test[['X_14','X_15','X_16','X_17','X_18']].mean(axis=1)
test['X19~X22'] = test[['X_19','X_20','X_21','X_22']].mean(axis=1)
test['X24~X29'] = test[['X_24','X_25','X_26','X_27','X_28','X_29']].mean(axis=1)
test['X30~X32'] = test[['X_30','X_32']].mean(axis=1)
test['X31~X33'] = test[['X_31','X_33']].mean(axis=1)
test['X38~X40'] = test[['X_38','X_39','X_40']].mean(axis=1)
test['X41~X45'] = test[['X_41','X_42','X_43','X_44','X_45']].mean(axis=1)
test['X50~X56'] = test[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']].mean(axis=1)

In [ ]:
test['con1_X'] = test['X_12'] - test['X_24']
test['con2_X'] = test['X_12'] - test['X_25']
test['con3_X'] = test['X_12'] - test['X_26']
test['con4_X'] = test['X_12'] - test['X_27']
test['con5_X'] = test['X_12'] - test['X_28']
test['con6_X'] = test['X_12'] - test['X_29']

In [ ]:
test['nd_area_X1'] = test['X_08']*test['X_07']/test['X_03']
test['th_area_X1'] = test['X_09']*test['X_07']/test['X_03']

test['nd_area_X2'] = test['X_08']*test['X_07']/test['X_03']
test['th_area_X2'] = test['X_09']*test['X_07']/test['X_03']

In [ ]:
# 무게 / 면적
test['X3/X7'] = test['X_03']/ test['X_07']

# 스크류 깊이 평균
test['X19X30'] = test[['X_19','X_30']].mean(axis=1)
test['X20X31'] = test[['X_20','X_31']].mean(axis=1)
test['X21X32'] = test[['X_21','X_32']].mean(axis=1)
test['X22X33'] = test[['X_22','X_33']].mean(axis=1)

In [ ]:
# 표준편차
test['X14~X18_std'] = test[['X_14', 'X_15', 'X_16', 'X_17', 'X_18']].std(axis=1)
test['X19~X22_std'] = test[['X_19','X_20','X_21','X_22']].std(axis=1)
test['X24~X29_std'] =  test[['X_24', 'X_25', 'X_26','X_27','X_28','X_29']].std(axis=1)

test['X38~X40_std'] = test[['X_38','X_39','X_40']].std(axis=1)

test['X41~X45_std'] = test[['X_41','X_42', 'X_43', 'X_44', 'X_45']].std(axis=1)
test['X50~X56_std'] = test[['X_50','X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56']].std(axis=1)

In [ ]:
test['X14~X18_mid'] = test[['X_14', 'X_15', 'X_16', 'X_17', 'X_18']].median(axis=1)
test['X19~X22_mid'] = test[['X_19','X_20','X_21','X_22']].median(axis=1)
test['X24~X29_mid'] =  test[['X_24', 'X_25', 'X_26','X_27','X_28','X_29']].median(axis=1)

test['X38~X40_mid'] = test[['X_38','X_39','X_40']].median(axis=1)

test['X41~X45_mid'] = test[['X_41','X_42', 'X_43', 'X_44', 'X_45']].median(axis=1)
test['X50~X56_mid'] = test[['X_50','X_51', 'X_52', 'X_53', 'X_54', 'X_55', 'X_56']].median(axis=1)

In [ ]:
test['gongcha_1_X'] = test[['X_38','X_39','X_40']].sum(axis = 1)
test['gongcha_2_X'] = test[['X_24','X_25','X_26','X_27','X_28','X_29']].sum(axis = 1)

In [ ]:
test['all_sum_X'] = test.sum(axis=1)
test['all_mean_X'] = test.mean(axis=1)

In [ ]:
train_x = train.filter(regex='X') # Input : X Featrue
train_y = train.filter(regex='Y') # Output : Y Feature

test_x = test.drop(columns=['ID'])

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)

(39607, 92)
(39607, 14)
(39608, 92)


## 모델링

In [ ]:
from pycaret.regression import *

'cuml' is a soft dependency and not included in the pycaret installation. Please run: `pip install cuml` to install.
'cuml' is a soft dependency and not included in the pycaret installation. Please run: `pip install cuml` to install.
'cuml' is a soft dependency and not included in the pycaret installation. Please run: `pip install cuml` to install.
'cuml' is a soft dependency and not included in the pycaret installation. Please run: `pip install cuml` to install.


In [ ]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(SEED) # Seed 고정

In [ ]:
train_01 = pd.concat([train_x, train_y['Y_01']], axis=1)
train_02 = pd.concat([train_x, train_y['Y_02']], axis=1)
train_03 = pd.concat([train_x, train_y['Y_03']], axis=1)
train_04 = pd.concat([train_x, train_y['Y_04']], axis=1)
train_05 = pd.concat([train_x, train_y['Y_05']], axis=1)
train_06 = pd.concat([train_x, train_y['Y_06']], axis=1)
train_07 = pd.concat([train_x, train_y['Y_07']], axis=1)
train_08 = pd.concat([train_x, train_y['Y_08']], axis=1)
train_09 = pd.concat([train_x, train_y['Y_09']], axis=1)
train_10 = pd.concat([train_x, train_y['Y_10']], axis=1)
train_11 = pd.concat([train_x, train_y['Y_11']], axis=1)
train_12 = pd.concat([train_x, train_y['Y_12']], axis=1)
train_13 = pd.concat([train_x, train_y['Y_13']], axis=1)
train_14 = pd.concat([train_x, train_y['Y_14']], axis=1)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
def lg_nrmse_py(gt, preds):
    all_nrmse = []
    rmse = mean_squared_error(gt, preds, squared=False)
    nrmse = rmse/np.mean(np.abs(gt[:]))
    score = nrmse
    return score

## Y_06

In [ ]:
model_Y_06 = setup(train_06, target = 'Y_06', session_id=SEED )

INFO:logs:PyCaret RegressionExperiment
INFO:logs:Logging name: reg-default-name
INFO:logs:ML Usecase: MLUsecase.REGRESSION
INFO:logs:version 3.0.0.rc3
INFO:logs:Initializing setup()
INFO:logs:self.USI: 2069
INFO:logs:self.variable_keys: {'USI', 'y', 'fold_shuffle_param', 'fold_groups_param', 'X_train', '_all_models', 'idx', 'exp_id', 'X_test', 'variable_keys', 'pipeline', 'log_plots_param', '_gpu_n_jobs_param', 'y_test', 'gpu_param', '_all_metrics', 'exp_name_log', 'transform_target_param', 'logging_param', 'master_model_container', 'seed', 'target_param', 'memory', 'y_train', 'X', '_all_models_internal', 'fold_generator', 'display_container', 'data', 'html_param', 'transform_target_method_param', '_ml_usecase', '_available_plots', 'n_jobs_param'}
INFO:logs:Checking environment
INFO:logs:python_version: 3.7.13
INFO:logs:python_build: ('default', 'Apr 24 2022 01:04:09')
INFO:logs:machine: x86_64
INFO:logs:platform: Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
INFO:logs:Memory: svmem(t

,Description,Value
0,Session id,42
1,Target,Y_06
2,Target type,Regression
3,Data shape,"(39607, 93)"
4,Train data shape,"(27724, 93)"
5,Test data shape,"(11883, 93)"
6,Numeric features,92
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


INFO:logs:Soft dependency imported: xgboost: 1.1.0
INFO:logs:Soft dependency imported: catboost: 1.0.6
INFO:logs:Soft dependency imported: xgboost: 1.1.0
INFO:logs:Soft dependency imported: catboost: 1.0.6
INFO:logs:setup() successfully completed in 6.0s...............


In [ ]:
model_Y_06.add_metric('lg_nrmse_py', 'lg_nrmse_py', lg_nrmse_py, greater_is_better=False)

Name                                                       lg_nrmse_py
Display Name                                               lg_nrmse_py
Score Function                <function lg_nrmse_py at 0x7f5318308560>
Scorer               make_scorer(lg_nrmse_py, greater_is_better=False)
Target                                                            pred
Args                                                                {}
Greater is Better                                                False
Custom                                                            True
Name: lg_nrmse_py, dtype: object

In [ ]:
cat_Y_06 = create_model('catboost', cross_validation = True, fold = 10, round=5)

INFO:logs:Initializing create_model()
INFO:logs:create_model(self=<pycaret.regression.oop.RegressionExperiment object at 0x7f531ef2d2d0>, estimator=catboost, fold=10, round=5, cross_validation=True, predict=True, fit_kwargs=None, groups=None, refit=True, probability_threshold=None, experiment_custom_tags=None, verbose=True, system=True, add_to_model_list=True, metrics=None, display=None, model_only=True, return_train_score=False, kwargs={})
INFO:logs:Checking exceptions


,MAE,MSE,RMSE,R2,RMSLE,MAPE,lg_nrmse_py
Fold,,,,,,,
0,0.74102,3.46684,1.86195,0.12059,0.06292,0.04491,0.11212
1,0.72812,2.57061,1.60331,0.05072,0.06307,0.04424,0.09644
2,0.74134,2.80142,1.67374,-0.16797,0.07416,0.04478,0.10071
3,0.72715,3.77231,1.94224,0.12609,0.05605,0.04383,0.11665
4,0.72331,2.48202,1.57544,0.22816,0.07317,0.04388,0.09505
5,0.76884,4.41764,2.10182,0.09645,0.07588,0.04616,0.12644
6,0.72126,2.62389,1.61984,0.18041,0.08058,0.04359,0.09750
7,0.74773,3.06111,1.74960,0.22323,0.07449,0.04540,0.10533
8,0.76330,3.85880,1.96438,0.05191,0.08228,0.04608,0.11827


INFO:logs:Importing libraries
INFO:logs:Copying training dataset
INFO:logs:Defining folds
INFO:logs:Declaring metric variables
INFO:logs:Importing untrained model
INFO:logs:CatBoost Regressor Imported successfully
INFO:logs:Starting cross validation
INFO:logs:Cross validating with KFold(n_splits=10, random_state=None, shuffle=False), n_jobs=-1
INFO:logs:Calculating mean and std
INFO:logs:Creating metrics dataframe
INFO:logs:Finalizing model
INFO:logs:Uploading results into container
INFO:logs:Uploading model into container now
INFO:logs:master_model_container: 1
INFO:logs:display_container: 2
INFO:logs:<catboost.core.CatBoostRegressor object at 0x7f53179cbe10>
INFO:logs:create_model() successfully completed......................................


In [ ]:
xgb_Y_06 = create_model('xgboost', cross_validation = True, fold = 10, round=5)

INFO:logs:Initializing create_model()
INFO:logs:create_model(self=<pycaret.regression.oop.RegressionExperiment object at 0x7f531ef2d2d0>, estimator=xgboost, fold=10, round=5, cross_validation=True, predict=True, fit_kwargs=None, groups=None, refit=True, probability_threshold=None, experiment_custom_tags=None, verbose=True, system=True, add_to_model_list=True, metrics=None, display=None, model_only=True, return_train_score=False, kwargs={})
INFO:logs:Checking exceptions


,MAE,MSE,RMSE,R2,RMSLE,MAPE,lg_nrmse_py
Fold,,,,,,,
0,0.79533,3.91660,1.97904,0.00650,0.08991,0.04817,0.11917
1,0.76454,2.83276,1.68308,-0.04609,0.07001,0.04648,0.10124
2,0.78483,2.99864,1.73166,-0.25020,0.10552,0.04748,0.10420
3,0.75467,4.13273,2.03291,0.04259,0.06314,0.04553,0.12210
4,0.77776,2.94946,1.71740,0.08279,0.09981,0.04718,0.10362
5,0.78291,4.67525,2.16223,0.04376,0.08201,0.04708,0.13008
6,0.75941,2.91691,1.70790,0.08888,0.07332,0.04592,0.10280
7,0.79313,3.47874,1.86514,0.11726,0.09165,0.04816,0.11229
8,0.81212,4.28489,2.07000,-0.05278,0.09622,0.04905,0.12463


INFO:logs:Importing libraries
INFO:logs:Copying training dataset
INFO:logs:Defining folds
INFO:logs:Declaring metric variables
INFO:logs:Importing untrained model
INFO:logs:Extreme Gradient Boosting Imported successfully
INFO:logs:Starting cross validation
INFO:logs:Cross validating with KFold(n_splits=10, random_state=None, shuffle=False), n_jobs=-1
INFO:logs:Calculating mean and std
INFO:logs:Creating metrics dataframe
INFO:logs:Finalizing model
INFO:logs:Uploading results into container
INFO:logs:Uploading model into container now
INFO:logs:master_model_container: 2
INFO:logs:display_container: 3
INFO:logs:XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_

In [ ]:
gbr_Y_06 = create_model('gbr', cross_validation = True, fold = 10, round=5)

INFO:logs:Initializing create_model()
INFO:logs:create_model(self=<pycaret.regression.oop.RegressionExperiment object at 0x7f531ef2d2d0>, estimator=gbr, fold=10, round=5, cross_validation=True, predict=True, fit_kwargs=None, groups=None, refit=True, probability_threshold=None, experiment_custom_tags=None, verbose=True, system=True, add_to_model_list=True, metrics=None, display=None, model_only=True, return_train_score=False, kwargs={})
INFO:logs:Checking exceptions


,MAE,MSE,RMSE,R2,RMSLE,MAPE,lg_nrmse_py
Fold,,,,,,,
0,0.75110,3.83781,1.95903,0.02648,0.06371,0.04550,0.11796
1,0.73205,2.80840,1.67583,-0.03710,0.07051,0.04443,0.10080
2,0.72245,2.47316,1.57263,-0.03111,0.05907,0.04362,0.09463
3,0.72331,4.03435,2.00857,0.06538,0.05107,0.04357,0.12063
4,0.72363,3.11246,1.76422,0.03210,0.06292,0.04384,0.10644
5,0.75578,4.70829,2.16986,0.03700,0.05596,0.04534,0.13054
6,0.72761,3.10928,1.76332,0.02879,0.05201,0.04389,0.10614
7,0.74765,3.69594,1.92248,0.06214,0.06251,0.04534,0.11574
8,0.74866,3.95026,1.98753,0.02943,0.05387,0.04513,0.11967


INFO:logs:Importing libraries
INFO:logs:Copying training dataset
INFO:logs:Defining folds
INFO:logs:Declaring metric variables
INFO:logs:Importing untrained model
INFO:logs:Gradient Boosting Regressor Imported successfully
INFO:logs:Starting cross validation
INFO:logs:Cross validating with KFold(n_splits=10, random_state=None, shuffle=False), n_jobs=-1
INFO:logs:Calculating mean and std
INFO:logs:Creating metrics dataframe
INFO:logs:Finalizing model
INFO:logs:Uploading results into container
INFO:logs:Uploading model into container now
INFO:logs:master_model_container: 3
INFO:logs:display_container: 4
INFO:logs:GradientBoostingRegressor(random_state=42)
INFO:logs:create_model() successfully completed......................................


In [ ]:
cat_Y_06_optuna = tune_model(cat_Y_06, fold=10, search_library='optuna',optimize='lg_nrmse_py',round=5, n_iter=50, early_stopping=True)

INFO:logs:Initializing tune_model()
INFO:logs:tune_model(estimator=<catboost.core.CatBoostRegressor object at 0x7f53179cbe10>, fold=10, round=5, n_iter=50, custom_grid=None, optimize=lg_nrmse_py, custom_scorer=None, search_library=optuna, search_algorithm=None, early_stopping=True, early_stopping_max_iters=10, choose_better=True, fit_kwargs=None, groups=None, return_tuner=False, verbose=True, tuner_verbose=True, return_train_score=False, kwargs={}, self=<pycaret.regression.oop.RegressionExperiment object at 0x7f531ef2d2d0>)
INFO:logs:Checking exceptions
INFO:logs:Soft dependency imported: optuna: 2.10.1


,MAE,MSE,RMSE,R2,RMSLE,MAPE,lg_nrmse_py
Fold,,,,,,,
0,0.73362,3.37425,1.83691,0.14407,0.06033,0.04449,0.11061
1,0.71725,2.41389,1.55367,0.10859,0.06143,0.04358,0.09345
2,0.73569,2.85087,1.68845,-0.18859,0.07585,0.04448,0.10160
3,0.70987,3.57300,1.89024,0.17226,0.07959,0.04281,0.11353
4,0.71686,2.40848,1.55193,0.25103,0.07964,0.04351,0.09364
5,0.74860,4.10192,2.02532,0.16103,0.07719,0.04502,0.12184
6,0.70428,2.38898,1.54563,0.25378,0.08093,0.04259,0.09303
7,0.72877,2.79932,1.67312,0.28966,0.07121,0.04428,0.10073
8,0.74816,3.58249,1.89275,0.11979,0.06892,0.04516,0.11396


INFO:logs:Copying training dataset
INFO:logs:Checking base model
INFO:logs:Base model : CatBoost Regressor
INFO:logs:Declaring metric variables
INFO:logs:Defining Hyperparameters
INFO:logs:Tuning with n_jobs=-1
INFO:logs:Initializing optuna.integration.OptunaSearchCV
INFO:logs:Soft dependency imported: xgboost: 1.1.0
INFO:logs:can_partial_fit: False, can_warm_start: False, is_xgboost: False
[I 2022-08-21 09:43:33,046] Searching the best hyperparameters using 27724 samples...
[I 2022-08-21 14:19:31,555] Finished hyperparemeter search!
INFO:logs:best_params: {'actual_estimator__eta': 0.3210740400316016, 'actual_estimator__depth': 10, 'actual_estimator__n_estimators': 277, 'actual_estimator__random_strength': 0.3085096812883462, 'actual_estimator__l2_leaf_reg': 31}
  File "/usr/local/lib/python3.7/dist-packages/pycaret/internal/pycaret_experiment/supervised_experiment.py", line 2686, in tune_model
    cv_results = model_grid.cv_results_
AttributeError: 'OptunaSearchCV' object has no attri

In [ ]:
cat_Y_06_optuna.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'RMSE',
 'iterations': 277,
 'sampling_frequency': 'PerTree',
 'leaf_estimation_method': 'Newton',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'bayesian_matrix_reg': 0.10000000149011612,
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 31,
 'random_strength': 0.3085096776485443,
 'rsm': 1,
 'boost_from_average': True,
 'model_size_reg': 0.5,
 'pool_metainfo_options': {'tags': {}},
 'subsample': 0.800000011920929,
 'use_best_model': False,
 'random_seed': 42,
 'depth': 10,
 'posterior_sampling': False,
 'border_count': 254,
 'classes_count': 0,
 'auto_class_weights': 'None',
 'sparse_features_conflict_fraction': 0,
 'leaf_estimation_backtracking': 'AnyImprovement',
 'best_model_min_trees': 1,
 'model_shrink_rate': 0,
 'min_data_in_leaf': 1,
 'loss_function': 'RMSE',
 'learning_rate': 0.3210740387439728,
 'score_function': 'Cos

In [ ]:
xgb_Y_06_optuna = tune_model(xgb_Y_06, fold=10, search_library='optuna',optimize='lg_nrmse_py',round=5, n_iter=30, early_stopping=True)

INFO:logs:Initializing tune_model()
INFO:logs:tune_model(estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='auto', validate_parameters=1, verbosity=0), fold=10, round=5, n_iter=30, custom_grid=None, optimize=lg_nrmse_py, custom_scorer=None, search_library=optuna, search_algorithm=None, early_stopping=True, early_stopping_max_iters=10, choose_better=True, fit_kwargs=None, groups=None, return_tuner=False, verbose=True, tuner_verbose=True, return_train_score=False, kwargs={}, self=<pycaret.regression.oop.Regre

,MAE,MSE,RMSE,R2,RMSLE,MAPE,lg_nrmse_py
Fold,,,,,,,
0,0.74994,3.31672,1.82119,0.15867,0.06953,0.04543,0.10966
1,0.73518,2.43092,1.55914,0.10230,0.06128,0.04468,0.09378
2,0.74810,2.68968,1.64002,-0.12139,0.08763,0.04520,0.09869
3,0.72753,3.64038,1.90798,0.15665,0.06182,0.04392,0.11459
4,0.73861,2.63017,1.62178,0.18209,0.08980,0.04478,0.09785
5,0.78310,4.36906,2.09023,0.10639,0.08209,0.04708,0.12575
6,0.73167,2.86461,1.69251,0.10522,0.06113,0.04417,0.10187
7,0.77001,3.35279,1.83106,0.14922,0.09038,0.04671,0.11024
8,0.78701,3.74835,1.93606,0.07904,0.10161,0.04754,0.11657


INFO:logs:Copying training dataset
INFO:logs:Checking base model
INFO:logs:Base model : Extreme Gradient Boosting
INFO:logs:Declaring metric variables
INFO:logs:Defining Hyperparameters
INFO:logs:Tuning with n_jobs=-1
INFO:logs:Initializing optuna.integration.OptunaSearchCV
INFO:logs:Soft dependency imported: xgboost: 1.1.0
INFO:logs:can_partial_fit: False, can_warm_start: False, is_xgboost: False
[I 2022-08-21 14:34:19,236] Searching the best hyperparameters using 27724 samples...
[I 2022-08-21 16:11:17,440] Finished hyperparemeter search!
INFO:logs:best_params: {'actual_estimator__learning_rate': 0.08973152175238867, 'actual_estimator__n_estimators': 217, 'actual_estimator__subsample': 0.7939653075858144, 'actual_estimator__max_depth': 7, 'actual_estimator__colsample_bytree': 0.7646370563887103, 'actual_estimator__min_child_weight': 4, 'actual_estimator__reg_alpha': 0.04559834983276881, 'actual_estimator__reg_lambda': 2.4611603561987767, 'actual_estimator__scale_pos_weight': 41.49088

Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


INFO:logs:master_model_container: 8
INFO:logs:display_container: 6
INFO:logs:XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7646370563887103, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.08973152175238867, max_delta_step=0, max_depth=7,
             min_child_weight=4, missing=nan, monotone_constraints='()',
             n_estimators=217, n_jobs=-1, num_parallel_tree=1, random_state=42,
             reg_alpha=0.04559834983276881, reg_lambda=2.4611603561987767,
             scale_pos_weight=41.49088008454724, subsample=0.7939653075858144,
             tree_method='auto', validate_parameters=1, verbosity=0)
INFO:logs:tune_model() successfully completed......................................


In [ ]:
xgb_Y_06_optuna

In [ ]:
gbr_Y_06_optuna = tune_model(gbr_Y_06, fold=10, search_library='optuna',optimize='lg_nrmse_py',round=5, n_iter=30, early_stopping=True)

INFO:logs:Initializing tune_model()
INFO:logs:tune_model(estimator=GradientBoostingRegressor(random_state=42), fold=10, round=5, n_iter=30, custom_grid=None, optimize=lg_nrmse_py, custom_scorer=None, search_library=optuna, search_algorithm=None, early_stopping=True, early_stopping_max_iters=10, choose_better=True, fit_kwargs=None, groups=None, return_tuner=False, verbose=True, tuner_verbose=True, return_train_score=False, kwargs={}, self=<pycaret.regression.oop.RegressionExperiment object at 0x7f531ef2d2d0>)
INFO:logs:Checking exceptions
INFO:logs:Soft dependency imported: optuna: 2.10.1


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:24:07
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Regressor


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

INFO:logs:Copying training dataset
INFO:logs:Checking base model
INFO:logs:Base model : Gradient Boosting Regressor
INFO:logs:Declaring metric variables
INFO:logs:Defining Hyperparameters
INFO:logs:Tuning with n_jobs=-1
INFO:logs:Initializing optuna.integration.OptunaSearchCV
INFO:logs:Soft dependency imported: xgboost: 1.1.0
INFO:logs:can_partial_fit: False, can_warm_start: False, is_xgboost: False
[I 2022-08-21 16:24:07,716] Searching the best hyperparameters using 27724 samples...


In [ ]:
gbr_Y_06_optuna